<a href="https://colab.research.google.com/github/WahyuKhairi06/TextMining_WahyuKhairi_2311531009/blob/main/Tugas_2_Text_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Library**

In [1]:
!pip install spacy
!python -m spacy download en_core_web_md
!pip install gensim
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

import spacy

/usr/local/lib/python3.12/dist-packages/cupyx/scipy/sparse/_base.py:10: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.3)
  import scipy.sparse as _sparse
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 27.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.3
    Uninstalling numpy-2.3.3:
      Successfully uninstalled numpy-2.3.3
ERROR: pip's depend

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

# **Prepocessing**

read dataset


In [ ]:


# Baca dataset
df = pd.read_csv("Pendidikan.csv")

# Lihat 10 data teratas
print(df.head(10))


In [ ]:
df.shape

In [ ]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Lowercasing
df['cleaned_text'] = df['full_text'].str.lower()

# Hapus angka, tanda baca, dan karakter khusus
df['cleaned_text'] = df['cleaned_text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

# Tokenisasi
df['tokens'] = df['cleaned_text'].apply(word_tokenize)

# Remove stopwords (optional, but common in text cleaning)
stop_words = set(stopwords.words('english')) # You might want to use Indonesian stopwords if the text is in Indonesian
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

# Display the first few rows with the new columns
display(df[['full_text', 'cleaned_text', 'tokens']].head())

In [ ]:


factory = StemmerFactory()
stemmer = factory.create_stemmer()

df['stemmed'] = df['tokens'].apply(lambda x: [stemmer.stem(word) for word in x])

print(df[['tokens', 'stemmed']].head())

# **1. BoW (bag of word)**

In [ ]:

df['stemmed_text'] = df['stemmed'].apply(lambda x: ' '.join(x))

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['stemmed_text'])

print("Shape of BoW matrix:", X.shape)
print("Feature names (first 50):", vectorizer.get_feature_names_out()[:50])

#**2. TF-IDF (Term Frequency – Inverse Document Frequency)**

In [ ]:


tfidf_vectorizer = TfidfVectorizer()

X_tfidf = tfidf_vectorizer.fit_transform(df['stemmed_text'])

print("Shape of TF-IDF matrix:", X_tfidf.shape)
print("Feature names (first 50):", tfidf_vectorizer.get_feature_names_out()[:50])

# **3. Word2Vec**

In [ ]:
model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Display the vocabulary size
print("Vocabulary size:", len(model.wv.index_to_key))

# You can access the vector for a word like this:
print("Vector for 'pendidikan':", model.wv['pendidikan'])

# You can find similar words like this:
print("Words similar to 'sekolah':", model.wv.most_similar('sekolah'))

# **4. GloVe**

In [ ]:

try:
    nlp = spacy.load("en_core_web_md")
except OSError:
    print("Downloading en_core_web_md model...")
    from spacy.cli import download
    download("en_core_web_md")
    nlp = spacy.load("en_core_web_md")


# Function to get the GloVe vector for a text
def get_glove_vector(text):
    # Process the text with the spaCy model
    doc = nlp(text)
    # Return the vector for the entire document (average of word vectors)
    # You can also get vectors for individual tokens: token.vector
    return doc.vector

# Apply the function to the 'stemmed_text' column to get GloVe embeddings for each document
df['glove_embeddings'] = df['stemmed_text'].apply(get_glove_vector)

# Display the shape of the resulting embeddings for the first few rows
print("Shape of GloVe embeddings for the first 5 documents:")
for i in range(min(5, len(df))):
    print(f"Document {i}: {df['glove_embeddings'][i].shape}")

# You can access the vector for a specific document like this:
# print("\nGloVe embedding for the first document:", df['glove_embeddings'][0])